In [1]:
import halide as hl
import numpy as np

# Vars
x, y = hl.Var("x"), hl.Var("y")

# Func
f = hl.Func("f")
f[x, y] = x + y

# Realize into a 10x10 buffer
output = f.realize([10, 10])

# Convert Buffer -> numpy (safe way)
shape = (output.dim(1).extent(), output.dim(0).extent())
out_np = np.zeros(shape, dtype=np.int32)

for j in range(shape[0]):
    for i in range(shape[1]):
        out_np[j, i] = output[i, j]

print("Halide output (10x10):")
print(out_np)


Halide output (10x10):
[[ 0  1  2  3  4  5  6  7  8  9]
 [ 1  2  3  4  5  6  7  8  9 10]
 [ 2  3  4  5  6  7  8  9 10 11]
 [ 3  4  5  6  7  8  9 10 11 12]
 [ 4  5  6  7  8  9 10 11 12 13]
 [ 5  6  7  8  9 10 11 12 13 14]
 [ 6  7  8  9 10 11 12 13 14 15]
 [ 7  8  9 10 11 12 13 14 15 16]
 [ 8  9 10 11 12 13 14 15 16 17]
 [ 9 10 11 12 13 14 15 16 17 18]]


In [22]:
import halide as hl
import imageio
import numpy as np

# Define input
input = hl.ImageParam(hl.Float(32), 3)
f = hl.Func('f')
x, y, c = hl.Var('x'), hl.Var('y'), hl.Var('c')

# Crop: output only the top-left 2x2 region
f[x, y, c] = hl.min(2 * input[x, y, c], 1.0)

# Load input image
img_np = imageio.imread('input.png').astype(np.float32) / 255.0
img_np = np.ascontiguousarray(img_np)
img_np = np.transpose(img_np, (2,0,1)).copy()
img = hl.Buffer(img_np)
input.set(img)

# Realize 2x2 crop
output = f.realize([100, 100, img.channels()])

output = np.array(output)
output = np.transpose(output, (1,2,0))  # [height, width, channels]

# Save output
imageio.imsave('output.png', (output * 255.0).astype(np.uint8))


C:\Users\Cezan\AppData\Local\Temp\ipykernel_17092\763385358.py:14: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_np = imageio.imread('input.png').astype(np.float32) / 255.0


In [46]:
A, B = hl.ImageParam(hl.Float(32), 2, "A"), hl.ImageParam(hl.Float(32), 2, "B")
C = hl.Func("C")
i, j = hl.Var("i"), hl.Var("j")
C[i, j] = 0.0
k = hl.RDom([(0, A.dim(1).extent())], "k") # inner loop
C[i, j] += A[i, k] * B[k, j]


In [53]:
input = hl.ImageParam(hl.Float(32), 2, 'in')
kernel = hl.ImageParam(hl.Float(32), 2, 'k')
x, y = hl.Var('x'), hl.Var('y')

bounded_input = hl.Func('input')
bounded_input[x, y] = input[
    hl.clamp(x, 0, input.dim(0).extent() - 1),
    hl.clamp(y, 0, input.dim(1).extent() - 1)
]

conv_output = hl.Func('conv_output')
r = hl.RDom([(0, kernel.dim(0).extent()), (0, kernel.dim(1).extent())])
# Initialize conv_output[x, y] to zero before reduction
conv_output[x, y] = 0.0
conv_output[x, y] += bounded_input[
    x - r.x + kernel.dim(0).extent() // 2,
    y - r.y + kernel.dim(1).extent() // 2
] * kernel[r.x, r.y]

In [4]:
# By convention, we import halide as 'hl' for terseness
import halide as hl

# Some constants
edge = 512
k = 20.0 / float(edge)

# Simple formula
x, y, c = hl.Var('x'), hl.Var('y'), hl.Var('c')
f = hl.Func('f')
e = hl.sin(x * ((c + 1) / 3.0) * k) * hl.cos(y * ((c + 1) / 3.0) * k)
f[x, y, c] = hl.cast(hl.UInt(8), e * 255.0)
f.vectorize(x, 8).parallel(y)

# Realize into a Buffer.
buf = f.realize([edge, edge, 3])

# Do something with the image. We'll just save it to a PNG.
from halide import imageio

imageio.imwrite("example.png", buf)

In [14]:
import halide as hl
import imageio
import numpy as np

# Constructing Halide functions statically.
input = hl.ImageParam(hl.Float(32), 3)
f = hl.Func('f')
x, y, c = hl.Var('x'), hl.Var('y'), hl.Var('c')
# Double the values and clamp them by 1.
f[x, y, c] = hl.min(2 * input[x, y, c], 1.0)

# Actually compiling/executing the Halide functions.
#
# Setup the input by loading an image (Halide assumes Fortran ordering).
img_np = imageio.imread('input.png').astype(np.float32) / 255.0
# Ensure shape is (height, width, channels) and memory is contiguous
img_np = np.ascontiguousarray(img_np)

img = hl.Buffer(img_np)
input.set(img)
# Process the input by calling f.realize
output = f.realize([img.width(), img.height(), img.channels()])
# Save the image to a file by converting to a numpy array.
output = np.array(output)
imageio.imsave('output.png', (output * 255.0).astype(np.uint8))


# import halide as hl
# import imageio
# import numpy as np

# # Constructing Halide functions statically.
# input = hl.ImageParam(hl.Float(32), 3)
# f = hl.Func('f')
# x, y, c = hl.Var('x'), hl.Var('y'), hl.Var('c')
# # Double the values and clamp them by 1.
# f[x, y, c] = hl.min(2 * input[x, y, c], 1.0)

# # Actually compiling/executing the Halide functions.
# #
# # Setup the input by loading an image (Halide assumes Fortran ordering).
# img = hl.Buffer(np.asfortranarray(imageio.imread('images/rgb.png').astype(np.float32) / 255.0))
# input.set(img)
# # Process the input by calling f.realize
# output = f.realize(img.width(), img.height(), img.channels())
# # Save the image to a file by converting to a numpy array.
# output = np.array(output)
# imageio.imsave('output.png', (output * 255.0).astype(np.uint8))


C:\Users\Cezan\AppData\Local\Temp\ipykernel_17092\3822312771.py:15: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_np = imageio.imread('input.png').astype(np.float32) / 255.0


In [17]:
import halide as hl
import imageio
import numpy as np

# Step 1: Load and normalize the input image
img_np = imageio.imread("input.png").astype(np.float32)

# Step 2: Ensure shape is (height, width, channels) and memory is contiguous
if img_np.ndim == 2:
    img_np = img_np[:, :, np.newaxis]  # grayscale fallback
img_np = np.ascontiguousarray(img_np)

# Step 3: Transpose to match Halide's layout: [channels, height, width]
input_buf = hl.Buffer(img_np)

# Step 4: Declare Halide Vars and ImageParam
x, y, c = hl.Var("x"), hl.Var("y"), hl.Var("c")
input = hl.ImageParam(hl.Float(32), 3)
input.set(input_buf)

# Step 5: Brighten the image
value = hl.cast(hl.Float(32), input[x, y, c])
value = hl.min(value * 1.5, 255.0)
value = hl.cast(hl.UInt(8), value)

# Step 6: Define the Halide function
brighter = hl.Func("brighter")
brighter[x, y, c] = value

# Step 7: Realize the output buffer
channels = input_buf.dim(0).extent()
width = input_buf.dim(1).extent()
height = input_buf.dim(2).extent()
output = brighter.realize([channels, width, height])

# Step 8: Convert Halide buffer to NumPy and transpose back
output_np = np.array(output)

# Step 9: Save the brightened image
imageio.imsave("brighter.png", output_np)
print("Brightened image saved as brighter.png")

# Step 10: Debug pixel values
print("Original pixel:", img_np[100, 100])
print("Brightened pixel:", output_np[100, 100])


C:\Users\Cezan\AppData\Local\Temp\ipykernel_17000\1212804804.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_np = imageio.imread("input.png").astype(np.float32)


RuntimeError: Unable to cast Python instance to C++ type (#define PYBIND11_DETAILED_ERROR_MESSAGES or compile in debug mode for details)

In [ ]:
import halide as hl
import imageio
import numpy as np
from IPython.display import HTML

# Step 1: Load and normalize the input image
img_np = imageio.imread("input.png").astype(np.float32) / 255.0

# Step 2: Ensure shape is (height, width, channels) and memory is contiguous
if img_np.ndim == 2:
    img_np = img_np[:, :, np.newaxis]  # grayscale fallback
img_np = np.ascontiguousarray(img_np)

# Now wrap the Halide buffer from that array
input_buf = hl.Buffer(img_np)

# Optional quick sanity prints (remove in production)
print("After patch input_buf.width(), height(), channels():", input_buf.width(), input_buf.height(), input_buf.channels())
print("input_buf.dim(0).stride, dim(1).stride, dim(2).stride:",
      input_buf.dim(0).stride(), input_buf.dim(1).stride(), input_buf.dim(2).stride())
print("img_for_halide.shape, img_for_halide.strides (bytes):", img_for_halide.shape, img_for_halide.strides)

# Also check NumPy strides/order
print("img_np.shape, img_np.strides (bytes):", img_np.shape, img_np.strides)

# Step 4: Declare Halide Vars and ImageParam
x, y, c = hl.Var("x"), hl.Var("y"), hl.Var("c")
input = hl.ImageParam(hl.Float(32), 3)
input.set(input_buf)

# Step 5: Clamp edges to avoid out-of-bounds access
clamped = hl.Func("clamped")
clamped[x, y, c] = input[
    hl.clamp(x, 0, input_buf.width() - 1),
    hl.clamp(y, 0, input_buf.height() - 1),
    hl.clamp(c,0, input_buf.channels()-1)
]

# Step 6: Horizontal blur
blur_x = hl.Func("blur_x")
blur_x[x, y, c] = (clamped[x , y-1, c] + clamped[x, y, c] + clamped[x , y+1, c]) / 3.0

# Step 7: Vertical blur
blur_y = hl.Func("blur_y")
blur_y[x, y, c] = (blur_x[x, y , c-1] + blur_x[x, y, c] + blur_x[x, y , c+1]) / 3.0

# Step 8: Realize the output buffer using correct shape
output = blur_y.realize([input_buf.width(), input_buf.height(), input_buf.channels()])

# Step 9: Convert Halide Buffer to NumPy array
output_np = np.array(output)

# Step 10: Save the blurred image
imageio.imsave("blurred.png", (output_np * 255.0).astype(np.uint8))
print("Blurred image saved as blurred.png")

# Step 11: Debug pixel values
print("Original pixel:", img_np[0, 0])
print("Blurred pixel:", output_np[0,0])


C:\Users\Cezan\AppData\Local\Temp\ipykernel_17000\3452342675.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_np = imageio.imread("input.png").astype(np.float32) / 255.0


RuntimeError: Unable to cast Python instance to C++ type (#define PYBIND11_DETAILED_ERROR_MESSAGES or compile in debug mode for details)

In [13]:
import halide as hl
import imageio
import numpy as np

# Step 1: Load and normalize the input image
img_np = imageio.imread("input.png").astype(np.float32) / 255.0

# Step 2: Ensure shape is (height, width, channels) and memory is contiguous
if img_np.ndim == 2:
    img_np = img_np[:, :, np.newaxis]  # grayscale fallback
img_np = np.ascontiguousarray(img_np)

# Step 3: Transpose to match Halide's layout: [width, height, channels]
img_for_halide = np.transpose(img_np, (2,0,1)).copy()
input_buf = hl.Buffer(img_for_halide)

# Step 4: Declare Halide Vars and ImageParam
x, y, c = hl.Var("x"), hl.Var("y"), hl.Var("c")
input = hl.ImageParam(hl.Float(32), 3)
input.set(input_buf)

# Step 5: Clamp edges to avoid out-of-bounds access
clamped = hl.Func("clamped")
clamped[x, y, c] = input[
    hl.clamp(x, 0, input_buf.width() - 1),
    hl.clamp(y, 0, input_buf.height() - 1),
    c
]

# Step 6: Horizontal blur (blur across X)
blur_x = hl.Func("blur_x")
blur_x[x, y, c] = (clamped[x - 1, y, c] + clamped[x, y, c] + clamped[x + 1, y, c]) / 3.0

# Step 7: Vertical blur (blur across Y)
blur_y = hl.Func("blur_y")
blur_y[x, y, c] = (blur_x[x, y - 1, c] + blur_x[x, y, c] + blur_x[x, y + 1, c]) / 3.0

# Step 8: Realize the output buffer using correct shape
output = blur_y.realize([input_buf.width(), input_buf.height(), input_buf.channels()])

# Step 9: Convert Halide Buffer to NumPy array and transpose back to HWC
output_np = np.array(output)
output_np = np.transpose(output_np, (1,2,0))  # [height, width, channels]

# Step 10: Save the blurred image
imageio.imsave("blurred.png", (output_np * 255.0).astype(np.uint8))
print("Blurred image saved as blurred.png")

# Step 11: Debug pixel values
print("Original pixel:", img_np[100, 100])
print("Blurred pixel:", output_np[100, 100])


Blurred image saved as blurred.png
Original pixel: [0.53333336 0.7294118  0.24705882]
Blurred pixel: [0.5272332  0.72592604 0.23834427]


C:\Users\Cezan\AppData\Local\Temp\ipykernel_17092\4485534.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_np = imageio.imread("input.png").astype(np.float32) / 255.0


In [11]:
import subprocess, tempfile, os, numpy as np, imageio
from typing import List

class HalideValidatorLoop:
    def __init__(self, generator, max_attempts=10):
        self.generator = generator  # DSPy module or callable that returns halide_code + test_case
        self.max_attempts = max_attempts

    def run(self, prompt: str):
        results = []
        for i in range(self.max_attempts):
            print(f"\n🔁 Attempt {i+1}")
            result = self.generator(prompt)
            halide_code = result.halide_code
            test_case = result.test_case

            validation = self._validate_code(halide_code, test_case)
            results.append({
                "attempt": i+1,
                "halide_code": halide_code,
                "test_case": test_case,
                "syntax_ok": validation["syntax_ok"],
                "correctness_index": validation["correctness_index"],
                "error": validation["error"],
                "expected_pixel": validation.get("expected_pixel"),
                "actual_pixel": validation.get("actual_pixel")
            })

            if validation["correctness_index"] == 1.0:
                print("✅ Perfect match achieved. Stopping early.")
                break

        return results

    def _validate_code(self, halide_code: str, test_case: str):
        with tempfile.TemporaryDirectory() as tmpdir:
            code_path = os.path.join(tmpdir, "halide_test.py")
            with open(code_path, "w") as f:
                f.write(halide_code)

            try:
                subprocess.check_output(["python", code_path], stderr=subprocess.STDOUT)
            except subprocess.CalledProcessError as e:
                return {
                    "syntax_ok": False,
                    "error": e.output.decode(),
                    "correctness_index": 0.0
                }

            try:
                output_img = imageio.imread(os.path.join(tmpdir, "blurred.png")).astype(np.float32) / 255.0
                expected = self._extract_expected_pixel(test_case)
                actual = output_img[1, 1]  # center pixel of 3x3

                error = np.linalg.norm(actual - expected)
                score = max(0.0, 1.0 - error / 10.0)

                return {
                    "syntax_ok": True,
                    "error": None,
                    "correctness_index": round(score, 3),
                    "expected_pixel": expected.tolist(),
                    "actual_pixel": actual.tolist()
                }

            except Exception as e:
                return {
                    "syntax_ok": True,
                    "error": f"Pixel check failed: {str(e)}",
                    "correctness_index": 0.0
                }

    def _extract_expected_pixel(self, test_case: str):
        import re
        match = re.search(r"\[([0-9.,\s]+)\]", test_case)
        if match:
            values = [float(v.strip()) for v in match.group(1).split(",")]
            return np.array(values)
        return np.array([0.0, 0.0, 0.0])


In [12]:
validator = HalideValidatorLoop(generator=HalidePipeline(), max_attempts=10)
results = validator.run("Give me the full code to make an image input.png 2 times bright using Halide in Python")

for r in results:
    print(f"\nAttempt {r['attempt']}:")
    print("✅ Syntax OK:", r["syntax_ok"])
    print("🧪 Correctness Index:", r["correctness_index"])
    print("🔍 Expected vs Actual:", r.get("expected_pixel"), r.get("actual_pixel"))
    if r["error"]:
        print("❌ Error:", r["error"])


NameError: name 'HalidePipeline' is not defined